# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Loading-required-modules" data-toc-modified-id="Loading-required-modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading required modules</a></div><div class="lev1 toc-item"><a href="#Memory-Network" data-toc-modified-id="Memory-Network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Memory Network</a></div><div class="lev2 toc-item"><a href="#Implementation-of-the-Loss-Function" data-toc-modified-id="Implementation-of-the-Loss-Function-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Implementation of the Loss Function</a></div><div class="lev1 toc-item"><a href="#Training-the-model" data-toc-modified-id="Training-the-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training the model</a></div>

# Loading required modules

In [1]:
require 'nn';
require 'nngraph';

In [2]:
data_loader = require 'utils.data_loader';
QUICKLOAD = false

if QUICKLOAD then
    x = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/sample.t7")
    y = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/label.t7")
    voc = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7")
    index = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7_index")
else 
    input_file = "/Users/david/Documents/MemoryNetwork/preprocessing/output.txt"
    out_vocab_file = "/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7"
    out_tensor_file = "/Users/david/Documents/MemoryNetwork/output_lua/data.t7"
    voc = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7")
    index = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7_index")
    x, y,f = data_loader.text_to_tensor(input_file,out_vocab_file,out_tensor_file)
end

saving /Users/david/Documents/MemoryNetwork/output_lua/vocab.t7	


saving its index in /Users/david/Documents/MemoryNetwork/output_lua/vocab.t7_index	


putting sample tensor into /Users/david/Documents/MemoryNetwork/output_lua/sample.t7...	


saving /Users/david/Documents/MemoryNetwork/output_lua/sample.t7	
putting label tensor into /Users/david/Documents/MemoryNetwork/output_lua/data.t7...	


saving /Users/david/Documents/MemoryNetwork/output_lua/data.t7	


In [3]:
-- Checking x
print(x:size())
-- Checking y
print(y:size())
-- Checking f
print(f:size())

 70
 58
[torch.LongStorage of size 2]

 70
 58
[torch.LongStorage of size 2]

 70
  9
[torch.LongStorage of size 2]



In [4]:
-- pre-requists

require 'nn';
require 'nngraph';
require 'models.inference_module'
require 'utils.OneHot'

# Memory Network

In [5]:
require 'math';
require 'nn';
require 'nngraph';

MemoryModule = {}

function MemoryModule.create_network(num_mem,longuest_sentence_size,debug) -- longest_sentence,voc_size,num_mem)
    local inputs = {}
    local outputs = {}
    table.insert(inputs, nn.Identity()())
    -- 
    local reshaped_input =  nn.Reshape(1,-1)(inputs[1])
    local mem_proj = nn.SplitTable(1)(reshaped_input):annotate{name="mem_proj"}
    -- Creating memory mapping :
    local mem_size =  math.floor(longuest_sentence_size/num_mem)
    if debug then print("num_mem = "..num_mem) print("mem_size = "..mem_size) end
    for i=1,num_mem do
        local mem_mapping = nn.NarrowTable(1+(i-1)*mem_size,mem_size)(mem_proj)
        table.insert(outputs, nn.JoinTable(1)(mem_mapping)) -- Joining to get a table of tensors
    end 
    return nn.gModule(inputs, outputs)
end

In [6]:
RepresentationModule = {}

function RepresentationModule.create_network(vocab_size) 
    local inputs = {}
    local outputs = {}
    table.insert(inputs, nn.Identity()())
    -- 
    local one_hot = OneHot.new(vocab_size)(inputs[1]) -- One Hot Encoding 
    -- local sum_proj = nn.Reshape(1,-1, false)(nn.Sum(1)(one_hot)) -- Sum to obtain the doc representation with a count
    -- table.insert(outputs, sum_proj)
    table.insert(outputs, one_hot)
    return nn.gModule(inputs, outputs)
end

In [ ]:
NUM_MEM = 10
VOCAB_SIZE = 58

-- Incoding input
local rep = RepresentationModule.create_network(58)
-- print(rep:forward(x[1]):size())

-- Setting memory
mem_mod = MemoryModule.create_network(NUM_MEM,VOCAB_SIZE)
local xrepr = rep:forward(x[1])
local a = mem_mod:forward(xrepr)

-- Infering on memory + input
inf_mod = InferenceGraph.create_network(VOCAB_SIZE,3*VOCAB_SIZE)

for i=1,100 do
    local j = torch.random(1,NUM_MEM)
    local score = inf_mod:forward{xrepr,a[j]}
    print("memory "..j.." score : "..score[1][1])
end

In [ ]:
--x[1]
print(f[1])

In [48]:
--x[1]
print(f[1])

 22
 23
 24
 25
 26
 27
 28
  0
  0
[torch.DoubleTensor of size 9]




stereo	
in	
the	
utility	
room.	
caden	
joined	


bad argument #2 to '?' (out of range at /Users/david/torch/pkg/torch/generic/Tensor.c:913)
stack traceback:
	[C]: at 0x05e2ebf0
	[C]: in function '__index'
	[string "for i=1,f:size(2) do ..."]:2: in main chunk
	[C]: in function 'xpcall'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/david/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/david/torch/install/share/lua/5.1/lzmq/poller.lua:80: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0105cbaa10: 

In [ ]:
function get_supporting_facts(indice, fact_tensor,entry_table)
    print(index[entry_table[indice][fact_tensor[indice][i]]])
end

## Implementation of the Loss Function

In [105]:
function gradUpdate(mlp, x, y, criterion, learningRate)
   local pred = mlp:forward(x)
   local err = criterion:forward(pred, y)
   local gradCriterion = criterion:backward(pred, y)
   mlp:zeroGradParameters()
   mlp:backward(x, gradCriterion)
   mlp:updateParameters(learningRate)
end


local xrepr = o:forward(x[1])
gradUpdate(mlpa, {{x, y}, {x, z}}, 1, crit, 0.01)

Columns 1 to 26
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Columns 27 to 52
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Columns 53 to 58
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0

# Training the model